# 人脸识别
这个练习的目的是利用主成分分析（PCA）和线性判别分析（LDA）进行图像的人脸识别。在编码前先阅读所有指令。

## 1. 导入数据  
下载face.zip并从face.zip提取图像。你会发现一些人脸图像来自CMU PIE数据库。这里有10个人，每个人在24种不同的照明条件下拍摄，共有240幅图像。为了方便起见，这些图像在训练和测试目录中被分成两个相等的集合。  
图像的文件名ppppp_xx_yy.bmp，这里ppppp表示人的身份；xx表示头部方向（在这个练习中,所有的方向都是正面的（xx = 27））; YY表示照明条件。所有图像都被裁剪并对齐，它们的高度、宽度分别为160和140像素。

## 2. 特征提取  
特征提取在人脸识别中起着重要作用。好的特性应该能够区分不同的用户。  
在这部分中，您将学习如何提取PCA特征和LDA特征。
### 2.1 PCA特征
这部分将指导您从图像的训练集学习PCA特征提取器。您还将学习如何用图像提取PCA特征。  
遵照指示：  
1. 读取数据。通过将参数设置为训练文件夹的路径，使用read_faces读取所有训练图像。现在你应该有一个$22400\times120$的脸矩阵，它的列是训练目录中所有的人脸图像，以及相应的标签范围从0到9。  
2. 训练PCA。使用mypca函数，计算PCA投影矩阵$W$，全局均值向量$m_e$和特征值向量。学习代码- mypca，理解这是如何实现的。特别是，注意内积技巧的使用避免出现“内存不足”错误。    
3. 选择的特征维度。通过公式$ W_e = W[:,:K]$只保留前$K(K=30)$特征脸。    
4. 投影图像。对于一个人脸图像x，通过公式$y = W_e^T(x-m_e)$将它投影到PCA空间中。y是x的PCA特征表示。 
5. 反向投影PCA特征。众所周知，PCA也经常被用作降维方法。所以我们可以在已学习的PCA空间由特征向量y重建图像x：$x = W_ey+m_e$。

In [ ]:
def myPCA(A):
    # function [W,LL,m]=mypca(A)
    # computes PCA of matrix A
    # A: D by N data matrix. Each column is a random vector
    # W: D by K matrix whose columns are the principal components in
    # decreasing order
    # LL: eigenvalues
    # m: mean of columns of A
    
    # Note: 'lambda' is a Python reserved word
    
    # compute mean, and substract mean from every column
    [r,c] = A.shape
    m = np.mean(A,1)
    A = A - np.tile(m,(c,1)).T
    B = np.dot(A.T, A)
    [d, v] = linalg.eig(B)
    
    # sort d in descending order
    order_index = np.argsort(d)
    order_index = order_index[::-1]
    d = d[order_index]
    v = v[:,order_index]
    
    # compute eigenvectors of scatter matrix
    W = np.dot(A,v)
    Wnorm = ComputeNorm(W)
    
    W1 = np.tile(Wnorm, (r,1))
    W2 = W/W1
    
    LL = d[0:-1]
    
    W = W2[:,0:1]
    
    return W,LL,m 

In [3]:
import os
import numpy as np
import scipy.linalg as linalg
import cv2
import operator

def ComputerNorm(x):
    # function r=ComputerNorm(x)
    # computes vector norms of x
    # x : d x m matrix, each column a vector
    # r : 1 x m matrix, each the corresponding norm (L2)
    [row, col] = x.shape
    r = np.zeros((1,col))
    
    for i in range(col):
        r[0,i] = linalg.norm(x[:,i])
    return r  

def myLDA(A, Labels):
    # function [W,m] = myLDA(A, Label)
    # computes LDA of matrix A
    # A: D by N data matrix. Each column is a random vector. 
    # W: D by K matrix whose columns are the principal components in
    # decreasing order
    # m: mean of each projection
    classLabels = np.unique(Labels)
    classNum = len(classLabels)
    dim, datanum = A.shape
    totalMean = np.mean(A,1)
    partition = [np.where(Labels == label)[0] for label in classLabels]
    classMean = [(np.mean(A[:,idx],1),len(idx)) for idx in partition]
    
    # compute the within-class scatter matrix
    W = np.zeros((dim,dim))
    for idx in partition:
        W += np.cov(A[:,idx],rowvar = 1)*len(idx)
        
    # compute the between-class scatter matrix
    B = np.zeros((dim,dim))
    for mu, class_size in classMean:
        offset = mu - totalMean
        B += np.outer(offset, offset)*class_size
    
    
def read_faces(directory):
    # function faces = read_faces(directory)
    # Browse the directory, read image files and store faces in a matrix
    # faces: face matrix in which each column is a column vector for 1 face image
    # idLabels: corresponding ids for face matrix
    
    A = [] # A will store list of image vectors
    Label = [] # Label will store list of identity label
    for f in os.listdir(directory):
        

### 2.2 LDA特征
这部分将指导您从一组图像训练中学习LDA特征提取器。您还将学习如何用提取器提取图像的LDA特征。
由于计算代价，在LDA特征提取前经常采用PCA来对输入降维。根据PCA投影的结果，按照指示进行操作：  
1. 降维。将输入(22400维)降到$K_1$ ($K_1=90$)维。通过$W_1 = W[:,:K_1]$ 只保留前$K_1$个特征脸，W是在PCA特征提取的结果。降低x的维度通过$x'=W_1^T(x-m_e)$。对所有的人脸图像进行降维。现在应该有一个$K_1\times 120$维的矩阵X。    
2. 训练LDA。在矩阵X和对应的labels上训练LDA，计算得到LDA的投影矩阵$W_f$和在LDA的空间C上的各类中心。     
3. 投影图像。对于一个给定的人脸图像x，通过式$y = W_f^TW_1^T(x-m_e)$投影到LDA空间。

## 3. 识别系统。
根据我们学习的内容，人脸识别系统具有两个部分：注册(enrollment)和识别(identification)。  
### 3.1 Enrollment Session
建立人脸识别系统的第一部分是注册所有用户。也就是说，我们需要将所有用户的面部信息存储在系统数据库中。为了简单起见，我们可以使用每个用户的平均特征来表示他们的身份。基于第二节中的特征提取方法。按照如下指示进行操作：    
1. 将所有训练图像投影到相应的特征空间中。
2. 对于每个人，计算所有特征向量的平均z，按列存储这些向量在numpy矩阵Z，矩阵Z的第i列对应着身份标签-i，检查你的Z矩阵应该是$D_f\times 10$维的矩阵，其中$D_f$是特征维度。在我们的LDA的任务中，Z是myLDA函数返回的类中心。  

### 3.2 Identification Session  
你的面部识别系统现在已准备就绪。 为了识别新的脸部图像x(re-shaped into a vector)，首先通过将其投影到特征空间中来提取其特征。 然后使用欧几里得距离度量来搜索最接近于y的模板。最近模板的索引揭示了x的身份。 例如，如果最近的模板是第2列，则预测的id是classLabel [1]。    

您现在可以评估识别系统的性能。 使用测试目录中的图像，如上所述,使用您的识别系统识别每个图像。  

混淆矩阵(confusion matrix)是评估灯光识别系统性能的有用方法。 $M\times M$的 混淆矩阵C中的每个元素$c_{ij}$是系统将来自人i的图像标识为人j的次数。 因此完美的识别系统应该产生一个对角线的混淆矩阵。 C中的任何非零对角线元素都表示一个错误。 识别系统的总体准确度可以通过迹线除以所有元素的总和来计算。
计算$10\times10$混淆矩阵。 首先，将混淆矩阵中的所有条目初始化为0.然后，对于每个测试图像，让预测的id为您的分类器输出的身份，并让实际id为测试图像的实际身份。 将1添加到混淆矩阵的实际第id行和预测的第id列中的条目。